INF8770 Technologies multimédias

Polytechnique Montréal

Importation des librairies

In [ ]:
from PIL import Image
import os
from skimage.feature.texture import local_binary_pattern
from skimage.feature import graycomatrix
import glob
import cv2
import numpy
import csv
import time

 Matrices de Co-occurrence (MC)

In [ ]:
def desc_CM(imagePath, distances = [1], angles = [0]):#, numpy.pi / 2, numpy.pi, 3 * numpy.pi / 4]):
  # image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
  image = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
  # nbChannels = 3
  # nbRows = len(image)
  # nbColumns = len(image[0])
  # oneChannelImage = []
  # threeChannelsCoMatrix = []
  
  # for channel in range(nbChannels):
  #   oneChannelImage = []
  #   for row in range(nbRows):
  #     oneChannelImage.append([])
  #     for column in range(nbColumns):
  #       oneChannelImage[row].append(image[row][column][channel])
  #   cm = graycomatrix(oneChannelImage, distances, angles)
  #   threeChannelsCoMatrix.append(numpy.asarray(cm, numpy.float32))
  
  # for row in range(nbRows):
  #   oneChannelImage.append([])
  #   for column in range(nbColumns):
  #     oneChannelImage[row].append(image[row][column])
  #   cm = graycomatrix(oneChannelImage, distances, angles)
  #   threeChannelsCoMatrix.append(numpy.asarray(cm, numpy.float32))

  cm = graycomatrix(image, distances, angles)
  result = numpy.asarray(cm, numpy.float32)
  return result
  #return numpy.asarray(cm, numpy.float32)


  # threeChannelsSum = []
  # for row in range(256):
  #   threeChannelsSum.append([])
  #   for colum in range(256):
  #     sum = 0
  #     for channel in range(nbChannels):
  #       sum += threeChannelsCoMatrix[channel][row][colum]
  #     threeChannelsSum[row].append(sum)

  # return threeChannelsSum

Co-Occurrence Matrix in Grayscale 

In [ ]:
def desc_grayscale_CM(imagePath, distances = [1], angles = [0]):#, numpy.pi / 2, numpy.pi, 3 * numpy.pi / 4]):
    image = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
    cm = graycomatrix(image, distances, angles, normed=True)

    # Make the cm descriptor into a 1 dimension array
    co_matrix = []
    for row in range(len(cm)):
        for col in range(len(cm[0])):
            co_matrix.append(cm[row][col][0][0])

    return numpy.asarray(co_matrix, numpy.float32)

Co-Occurence Matrix in Color For Each Channel (probablement à delete)

In [ ]:
def desc_color_CM(imagePath, distances = [1], angles = [0]):
    image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
    nbChannels = 3
    nbRows = len(image)
    nbColumns = len(image[0])
    oneChannelImage = []
    threeChannelsCoMatrix = []
    
    for channel in range(nbChannels):
      oneChannelImage = []
      for row in range(nbRows):
        oneChannelImage.append([])
        for column in range(nbColumns):
          oneChannelImage[row].append(image[row][column][channel])
      cm = graycomatrix(oneChannelImage, distances, angles)
      threeChannelsCoMatrix.append(numpy.asarray(cm, numpy.float32))
    
    for row in range(nbRows):
      oneChannelImage.append([])
      for column in range(nbColumns):
        oneChannelImage[row].append(image[row][column])
      cm = graycomatrix(oneChannelImage, distances, angles)
      threeChannelsCoMatrix.append(numpy.asarray(cm, numpy.float32))
    

Modèles Binaires Locaux (LBP)

In [ ]:
def desc_LBP(path, longueurCodes, rayon):
  image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  desc = local_binary_pattern(image, longueurCodes, rayon)
  return numpy.asarray(desc, numpy.float32)

Importation des images

In [ ]:
query = glob.glob(os.path.join("./data", "*.jpg"))
database = glob.glob(os.path.join("./data/database", "*.jpg"))

query_cropped = glob.glob(os.path.join("./data/cropped", "*.jpg"))
database_cropped = glob.glob(os.path.join("./data/cropped/database", "*.jpg"))

Méthode de comparaison

In [ ]:
def intersection(vector_i, vector_j):
    return numpy.sum(numpy.minimum(vector_i, vector_j)) / numpy.sum(vector_j)

def norms1(vector_i, vector_j):
    return numpy.sum(numpy.abs(vector_i - vector_j))

def norms2(vector_i, vector_j):
    return numpy.sqrt(numpy.sum(numpy.power(vector_i - vector_j, 2)))

def bhattacharyya(vector_i, vector_j):
    return -numpy.log((numpy.sum(numpy.sqrt(numpy.multiply(vector_i, vector_j)))))

Classification par matrice de co-occurrence

In [80]:
def classify_MC(query, database):
    timeList = []
    
    for path_query in query:
        result = dict()

        timestamp = time.time_ns()
        cm_query = desc_grayscale_CM(path_query)
        timeList.append(time.time_ns() - timestamp)
    
        methods = ["intersection", "norms1", "norms2", "bhattacharyya"]

        path = path_query.split('\\')[1][:-4]
        with open(f"./results/MC/{path}.csv", 'w+', newline='') as file:
            writer = csv.writer(file)
            field = [path] + [x.split('\\')[1][:-4] for x in database]
            
            for key in methods:
                result[key] = []

            for path_db in database:
                timestamp = time.time_ns()
                cm_db = desc_grayscale_CM(path_db)
                timeList.append(time.time_ns() - timestamp)
                result_id = path_query.split('\\')[1][:-4] + ":" + path_db.split('\\')[1][:-4]

                result["intersection"].append((result_id, intersection(cm_query, cm_db)))
                result["norms1"].append((result_id, norms1(cm_query, cm_db)))
                result["norms2"].append((result_id, norms2(cm_query, cm_db)))
                result["bhattacharyya"].append((result_id, bhattacharyya(cm_query, cm_db)))
                
            writer.writerow([path + f" ({timeList[0]})"] + methods + ["Time"])

            for f in range(1, len(field)):
                writer.writerow([field[f]] + [result[key][f - 1][1] for key in result.keys()] + [timeList[f]])

        break

classify_MC(query, database)
classify_MC(query_cropped, database_cropped)

Classification par modèle binaire locaux

In [81]:
def classify_LBP(query, database, longueurCodes, rayon):
    timeList = []

    for path_query in query:
        result = dict()

        timestamp = time.time_ns()
        lbp_query = desc_LBP(path_query, longueurCodes, rayon)
        timeList.append(time.time_ns() - timestamp)

        row_lbp, col_lbp = cv2.imread(path_query, cv2.IMREAD_GRAYSCALE).shape

        lbp_hist_query, _ = numpy.histogram(lbp_query, bins = range(256))
        lbp_hist_query = lbp_hist_query / (row_lbp * col_lbp)

        methods = ["intersection", "norms1", "norms2", "bhattacharyya"]

        path = path_query.split('\\')[1][:-4]
        result_path = f"./results/LBP/R{rayon}-P{longueurCodes}"

        os.makedirs(result_path, exist_ok=True)

        with open(f"{result_path}/{path}.csv", 'w+', newline='') as file:
            writer = csv.writer(file)
            field = [path] + [x.split('\\')[1][:-4] for x in database]
            
            for key in methods:
                result[key] = []

            for path_db in database:
                timestamp = time.time_ns()
                lbp_db = desc_LBP(path_db, longueurCodes, rayon)
                
                row_lbp, col_lbp = cv2.imread(path_db, cv2.IMREAD_GRAYSCALE).shape

                lbp_hist_db, _ = numpy.histogram(lbp_db, bins = range(256))
                lbp_hist_db = lbp_hist_db / (row_lbp * col_lbp)

                result_id = path_query.split('\\')[1][:-4] + ":" + path_db.split('\\')[1][:-4]

                result["intersection"].append((result_id, intersection(lbp_hist_query, lbp_hist_db)))
                result["norms1"].append((result_id, norms1(lbp_hist_query, lbp_hist_db)))
                result["norms2"].append((result_id, norms2(lbp_hist_query, lbp_hist_db)))
                result["bhattacharyya"].append((result_id, bhattacharyya(lbp_hist_query, lbp_hist_db)))
                
                timeList.append(time.time_ns() - timestamp)

            writer.writerow([path + f" ({timeList[0]})"] + methods + ["Time"])

            for f in range(1, len(field)):
                writer.writerow([field[f]] + [result[key][f - 1][1] for key in result.keys()] + [timeList[f]])

for rayon in range(1, 6, 2):
    for longueurCodes in range(4, 25, 4):
        timestamp = time.time()
        classify_LBP(query, database, longueurCodes, rayon)
        print(f"Time Normal R{rayon}-P{longueurCodes}: {time.time() - timestamp}")
        timestamp = time.time()
        classify_LBP(query_cropped, database_cropped, longueurCodes, rayon)
        print(f"Time Cropped R{rayon}-P{longueurCodes}: {time.time() - timestamp}")

Time Normal R1-P4: 54928927600
Time Cropped R1-P4: 21719798500


KeyboardInterrupt: 